In [1]:
import pandas as pd
import numpy as np

income_df = pd.read_excel('ny_zip_income.xls',header=3,usecols=[0,1,2,16,17,18])

### Explore Income dataset

In [2]:
income_df.head().T

,0,1,2,3,4
ZIP\ncode [1],NaN,NaN,0,0,0
Size of adjusted gross income,NaN,NaN,Total,"$1 under $25,000","$25,000 under $50,000"
Number of returns,NaN,-1,9.46806e+06,3.44531e+06,2.12396e+06
Adjusted gross income (AGI) [4],NaN,-15,7.84411e+08,4.19902e+07,7.72745e+07
Total income,Number of returns,-16,9468050,3445290,2123960
Unnamed: 18,Amount,-17,795119743,42897756,78326525


### Data Cleaning - Income dataset

In [3]:
#rename columns
income_df = income_df.rename(columns={income_df.columns[0]: "zip_code", 
                                      income_df.columns[1]: "agi_bucket", 
                                      income_df.columns[2]: "num_returns", 
                                      income_df.columns[3]: "agi", 
                                      income_df.columns[4]: "tot_inc_num_returns", 
                                      income_df.columns[5]: "tot_inc_amnt"})
#only take the meaningful rows, avoiding totals
income_df = income_df.iloc[3:12329]

In [4]:
#change dollars (in thousands) columns to exact dollars
income_df['agi'] = income_df['agi']*1000
income_df['tot_inc_amnt'] = income_df['tot_inc_amnt']*1000

In [5]:
#drop rows that don't contain an agi_bucket value, they are pointless
income_df.dropna(subset=['agi_bucket'],inplace=True)

In [6]:
income_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9246 entries, 3 to 12328
Data columns (total 6 columns):
zip_code               9246 non-null object
agi_bucket             9246 non-null object
num_returns            9246 non-null float64
agi                    9246 non-null float64
tot_inc_num_returns    9246 non-null object
tot_inc_amnt           9246 non-null object
dtypes: float64(2), object(4)
memory usage: 505.6+ KB


In [7]:
#remove Zipcodes 0 & 99999
income_df = income_df.iloc[6:-6] 

In [8]:
#reset dataframe index
income_df.reset_index(drop=True,inplace=True)

In [9]:
income_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9234 entries, 0 to 9233
Data columns (total 6 columns):
zip_code               9234 non-null object
agi_bucket             9234 non-null object
num_returns            9234 non-null float64
agi                    9234 non-null float64
tot_inc_num_returns    9234 non-null object
tot_inc_amnt           9234 non-null object
dtypes: float64(2), object(4)
memory usage: 432.9+ KB


In [10]:
income_df.head()

,zip_code,agi_bucket,num_returns,agi,tot_inc_num_returns,tot_inc_amnt
0,10001,"$1 under $25,000",3740.0,43182000.0,3740,44849000
1,10001,"$25,000 under $50,000",2500.0,92133000.0,2500,94324000
2,10001,"$50,000 under $75,000",1950.0,120922000.0,1950,123197000
3,10001,"$75,000 under $100,000",1410.0,121880000.0,1410,123605000
4,10001,"$100,000 under $200,000",2580.0,361453000.0,2580,366195000


In [11]:
#show rows that have a num_returns discrepancy
income_df[income_df['tot_inc_num_returns'] != income_df['num_returns']]

,zip_code,agi_bucket,num_returns,agi,tot_inc_num_returns,tot_inc_amnt
1584,11355,"$1 under $25,000",34950.0,401891000.0,34940,412155000
7067,13833,"$200,000 or more",30.0,12181000.0,20,12181000
9105,14874,"$75,000 under $100,000",20.0,2656000.0,30,2656000


In [12]:
#convert zip_code column to integer datatype
income_df['zip_code'].astype('int64')

0       10001
1       10001
2       10001
3       10001
4       10001
5       10001
6       10002
7       10002
8       10002
9       10002
10      10002
11      10002
12      10003
13      10003
14      10003
15      10003
16      10003
17      10003
18      10004
19      10004
20      10004
21      10004
22      10004
23      10004
24      10005
25      10005
26      10005
27      10005
28      10005
29      10005
        ...  
9204    14898
9205    14898
9206    14898
9207    14898
9208    14898
9209    14898
9210    14901
9211    14901
9212    14901
9213    14901
9214    14901
9215    14901
9216    14903
9217    14903
9218    14903
9219    14903
9220    14903
9221    14903
9222    14904
9223    14904
9224    14904
9225    14904
9226    14904
9227    14904
9228    14905
9229    14905
9230    14905
9231    14905
9232    14905
9233    14905
Name: zip_code, Length: 9234, dtype: int64

### Filter New York State zip codes down to only New York City zip codes

In [13]:
#load data
nyc_zips_df = pd.read_csv('zip_borough.csv',usecols=[0]) #SOURCE: https://www.kaggle.com/kimjinyoung/nyc-borough-zip/downloads/nyc-borough-zip.zip/2


In [14]:
nyc_zips_df.head()

,zip
0,10001
1,10002
2,10003
3,10004
4,10005


In [15]:
#reset the income_df to only include NYC zip codes as seen in the nyc_zips_df
income_df = income_df[income_df['zip_code'].isin(nyc_zips_df['zip'])].reset_index(drop=True)

In [16]:
#convert numerical columns to float datatype
income_df['tot_inc_num_returns'] = income_df['tot_inc_num_returns'].astype('float64')
income_df['tot_inc_amnt'] = income_df['tot_inc_amnt'].astype('float64')

In [17]:
income_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1062 entries, 0 to 1061
Data columns (total 6 columns):
zip_code               1062 non-null object
agi_bucket             1062 non-null object
num_returns            1062 non-null float64
agi                    1062 non-null float64
tot_inc_num_returns    1062 non-null float64
tot_inc_amnt           1062 non-null float64
dtypes: float64(4), object(2)
memory usage: 49.9+ KB


In [24]:
income_df.describe()

,num_returns,agi,tot_inc_num_returns,tot_inc_amnt
count,1062.000000,1.062000e+03,1062.000000,1.062000e+03
mean,3785.969868,3.079024e+08,3785.960452,3.118991e+08
std,4365.667804,8.275105e+08,4365.600534,8.383446e+08
min,0.000000,0.000000e+00,0.000000,0.000000e+00
25%,1070.000000,8.402525e+07,1070.000000,8.529400e+07
50%,2400.000000,1.599225e+08,2400.000000,1.615430e+08
75%,4897.500000,2.712135e+08,4897.500000,2.753948e+08
max,36120.000000,9.954884e+09,36120.000000,1.008504e+10


### Explore Parks Dataset

In [18]:
parks_df = pd.read_json('DPR_Parks_001.json')

In [27]:
parks_df.head(30)

,Location,Name,Prop_ID,Zip
0,Franklin St. bet. Milton St. and Noble St.,American Playground,B001,11222
1,"E. 38 St., E. 38 St. bet. Ave. I and Ave. J",Amersfort Park,B002,11210
2,"Prospect Park W., 15 St.",Bartel-Pritchard Square,B003,11215
3,"Broadway, Stuyvesant Ave., Vernon Ave.",Beattie Square,B006,11221
4,Cropsey Ave. bet. 21 Ave. and Bay Pkwy.,Bensonhurst Park,B007,11214
5,"Blake Ave., Dumont Ave., Livonia Ave. bet. Str...",Betsy Head Park,B008,11212
6,"Washington Ave., Flatbush Ave. bet. Eastern Pk...",Brooklyn Botanic Garden,B010,11238
7,"St. Mark's Ave., Park Pl. bet. Brooklyn Ave. a...",Brower Park,B012,11213
8,"Irving Ave., Kinickerbocker Ave. bet. Starr St...",Maria Hernandez Park,B016,11237
9,Knickerbocker Ave. bet. Woodbine St. and Putna...,Bushwick Playground,B017,11237


In [20]:
parks_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1675 entries, 0 to 1674
Data columns (total 4 columns):
Location    1675 non-null object
Name        1675 non-null object
Prop_ID     1675 non-null object
Zip         1569 non-null object
dtypes: object(4)
memory usage: 52.4+ KB


In [63]:
parks_df.nunique()

Location    1669
Name        1532
Prop_ID     1675
Zip          278
dtype: int64

In [66]:
#explore duplicate park names
parks_df[parks_df.duplicated(subset=['Name'],keep=False)].sort_values(by=['Name'])

,Location,Name,Prop_ID,Zip
1376,Dr. MLK Jr. Blvd. bet. W. Tremont Ave. and E. ...,Aqueduct Walk,X001A,"10453, 10468"
1375,Aqueduct Ave. W. bet. W. Fordham Rd. and W. Ki...,Aqueduct Walk,X001,10468
713,bet. Hudson River and South End Ave. S/o Liber...,Battery Park City,M283A,10280
712,"Hudson River, Liberty St. West St, and Pier A",Battery Park City,M283,"10007, 10013, 10280"
312,Belmont Ave. between Forbell St. and Drew St.,Belmont Playground,B283,11208
1471,E 182 St bet. Belmont Av and Croton Av,Belmont Playground,X108,10457
1460,Harlem River bet. W 175 St and Alexander Hamil...,Bridge Park,X094,"10452, 10453"
220,"Prospect St., York St. bet. Jay St. and Bridge...",Bridge Park,B223JB,11201
550,"Broadway, Columbus Circle To W 110 St",Broadway Malls,M094,"10023, 10024, 10025"
552,"Broadway, W. 135 St. To W. 156 St.",Broadway Malls,M095,"10031, 10032"


- Insight: park names may span multiple Prop_ID's and multiple Zip Codes
- All 1,675 Prop_IDs are unique to each row - Keep all prop_id's

In [23]:
parks_df.describe()

,Location,Name,Prop_ID,Zip
count,1675,1675,1675,1569
unique,1669,1532,1675,278
top,Hoyt Ave. bet. 21 St. and 23 St.,Park,B280,11211
freq,2,36,1,41


### Explore Playgrounds dataset

In [28]:
playgrounds_df = pd.read_json('DPR_Playgrounds_001.json')

In [31]:
playgrounds_df.head(30)

,Accessible,Adaptive_Swing,Level,Location,Name,Playground_ID,Prop_ID,School_ID,Status,lat,lon
0,Y,N,4.0,"Noble, Franklin, Milton Streets",American Playground,B001,B001,None,None,40.7288,-73.9579
1,Y,N,4.0,BAY PKWY & CROPSEY AVENUE,Bensonhurst Park,B007-01,B007,None,None,40.5969,-73.9998
2,Y,N,1.0,DUMONT AVE & BRISTOL ST,Betsy Head Memorial Playground,B008-03,B008,None,None,40.6645,-73.9118
3,Y,N,2.0,BROOKLYN AVE & PROSPECT PL,Brower Park,B012-02,B012,None,None,40.6735,-73.9438
4,Y,Y,2.0,BROOKLYN AVE & PROSPECT PL,Brower Park,B012-03,B012,None,None,40.6744,-73.9432
5,Y,N,3.0,KNICKERBOCKER ST ENTRANCE,Maria Hernandez Park,B016-01,B016,None,None,40.7028,-73.9243
6,N,None,NaN,IRVING AVE ENTRANCE,Maria Hernandez Park,B016,B016,None,None,40.7036,-73.9234
7,Y,N,4.0,"Knickerbocker, Putnam Aves, Woodbine St",Bushwick Playground,B017,B017,None,None,40.6961,-73.9122
8,Y,N,4.0,"E. 92 ST, SEAVIEW AVE",Dinapoli Playground,B018-01,B018,None,None,40.6308,-73.8930
9,Y,Y,3.0,"Carroll, Court, President, Smith Sts",Carroll Park,B019,B019,None,None,40.6811,-73.9954


In [32]:
playgrounds_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1269 entries, 0 to 1268
Data columns (total 11 columns):
Accessible        1266 non-null object
Adaptive_Swing    1173 non-null object
Level             1175 non-null float64
Location          1266 non-null object
Name              1269 non-null object
Playground_ID     1021 non-null object
Prop_ID           1269 non-null object
School_ID         234 non-null object
Status            234 non-null object
lat               1245 non-null float64
lon               1245 non-null float64
dtypes: float64(3), object(8)
memory usage: 109.1+ KB


In [33]:
playgrounds_df.describe()

,Level,lat,lon
count,1175.000000,1245.000000,1245.000000
mean,3.571064,40.691978,-73.923549
std,0.792757,1.157509,0.090063
min,1.000000,0.000000,-74.244900
25%,3.000000,40.663300,-73.969800
50%,4.000000,40.717400,-73.925600
75%,4.000000,40.800700,-73.872000
max,4.000000,40.902500,-73.709600


In [59]:
print(playgrounds_df[~playgrounds_df['School_ID'].isna()].shape) #shape of non-null School_ID dataframe
print(playgrounds_df[playgrounds_df['School_ID'].isna()].shape) #shape of null School_ID dataframe
print(playgrounds_df.shape) #shape of dataframe

(234, 11)
(1035, 11)
(1269, 11)


In [60]:
#remove school playgrounds from playgrounds_df
playgrounds_df = playgrounds_df[playgrounds_df['School_ID'].isna()]

In [62]:
playgrounds_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1035 entries, 0 to 1268
Data columns (total 11 columns):
Accessible        1032 non-null object
Adaptive_Swing    939 non-null object
Level             941 non-null float64
Location          1032 non-null object
Name              1035 non-null object
Playground_ID     1021 non-null object
Prop_ID           1035 non-null object
School_ID         0 non-null object
Status            0 non-null object
lat               1018 non-null float64
lon               1018 non-null float64
dtypes: float64(3), object(8)
memory usage: 97.0+ KB


In [68]:
playgrounds_df.nunique()

Accessible           2
Adaptive_Swing       3
Level                4
Location          1027
Name               982
Playground_ID     1017
Prop_ID            865
School_ID            0
Status               0
lat                866
lon                835
dtype: int64

In [69]:
#explore duplicate Playground_ID's
playgrounds_df[playgrounds_df.duplicated(subset=['Playground_ID'],keep=False)].sort_values(by=['Playground_ID'])

,Accessible,Adaptive_Swing,Level,Location,Name,Playground_ID,Prop_ID,School_ID,Status,lat,lon
36,Y,N,2.0,"67 ST & COLONIAL RD, NORTH OF B066-02",Owls Head Park,B066,B066,None,None,40.6400,-74.0305
37,Y,N,2.0,"NR ENT @ COLONIAL RD/67 ST, SOUTH OF B066-01",Owls Head Park,B066,B066,None,None,40.6394,-74.0308
71,Y,N,4.0,"3 To 4 Sts, 4 To 5 Aves",Washington Park,B111,B111,None,None,40.6728,-73.9842
996,Y,N,4.0,Washington Park,Washington Park,B111,B111,None,None,NaN,NaN
572,Y,Y,3.0,"VERNON BLVD, 40 & 41 AVES",Vernon Playground,Q104-01,Q104,None,None,40.7571,-73.9470
574,N,None,NaN,,VERNON PLAYGROUND,Q104-01,Q104,None,None,40.7571,-73.9470
590,Y,N,4.0,225 St & 104 Ave,Pat Williams Playground,Q133,Q133,None,None,40.7100,-73.7285
591,Y,N,4.0,224 ST & 101 AVE,Pat Williams Playground,Q133,Q133,None,None,40.7100,-73.7285
1233,Y,N,4.0,At the corner of Cleveland and Mansion Avenues,Seaside Wildlife Nature Park,None,R145,None,None,40.5418,-74.1426
1237,Y,Y,3.0,Boardwalk between Beach 29th and Beach 30th St...,Beach 30th Street Playground,None,Q162,None,None,40.5926,-73.7620


- Some Prop_IDs may have multiple playgrounds - use a left join where all playgrounds remain in the set, and relevant zip codes are added to each

In [ ]:
#Impute Prop_ID as Playground ID

### Add Zip Codes from Parks dataset to Playground dataset using LEFT JOIN (playground = left df, parks = right df) on Prop_ID

In [ ]:
playgrounds_df.join(parks_df[['']])

### Join Parks & Playground datasets with Income by Zip Code dataset

In [ ]:


nyc_zips_df = pd.read_csv('zip_borough.csv') 